
# Brusselator - Using the `PDE` class

This example uses the :class:`~pde.pdes.pde.PDE` class to implement the
[Brusselator](https://en.wikipedia.org/wiki/Brusselator) with spatial
coupling,

\begin{align}\partial_t u &= D_0 \nabla^2 u + a - (1 + b) u + v u^2 \\
    \partial_t v &= D_1 \nabla^2 v + b u - v u^2\end{align}

Here, $D_0$ and $D_1$ are the respective diffusivity and the
parameters $a$ and $b$ are related to reaction rates.

Note that the same result can also be achieved with a 
:doc:`full implementation of a custom class <pde_brusselator_class>`, which
allows for more flexibility at the cost of code complexity.


In [12]:
import cv2
import os
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, UnitGrid, MemoryStorage

# Define the PDE
a, b = 1, 3
d0, d1 = 1, 0.1
eq = PDE(
    {
        "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
        "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
    }
)

# Initialize state
grid = UnitGrid([64, 64])
u = ScalarField(grid, a, label="Field $u$")
v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")
state = FieldCollection([u, v])

# Directory to save frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Create a MemoryStorage object to store the simulation state
storage = MemoryStorage()

# Simulate the PDE with storage tracker
sol = eq.solve(state, t_range=200, dt=1e-4, tracker=storage.tracker(interval=1))

# Save each state as an image
for frame_idx, (time, state) in enumerate(storage.items()):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    state[0].plot(ax=ax[0], vmin=0, vmax=5)
    state[1].plot(ax=ax[1], vmin=0, vmax=5)
    frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
    plt.savefig(frame_path)
    plt.close(fig)
    print(f"Saved frame {frame_idx} at time {time}")

# Custom directory to save the video file
video_directory = os.getcwd()
video_path = os.path.join(video_directory, 'brusselator.avi')

# Compile frames into a video
frame_rate = 24  # frames per second

# Get frame size from the first frame
first_frame = cv2.imread(os.path.join(frames_dir, 'frame_0000.png'))
if first_frame is None:
    raise ValueError("First frame not found. Check if the frames are being saved correctly.")
height, width, layers = first_frame.shape
frame_size = (width, height)

# Initialize video writer
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, frame_size)

# Write frames to video
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    if frame is None:
        print(f"Error reading frame {frame_file}. Skipping.")
        continue
    out.write(frame)

out.release()

print(f"Video saved to {video_path}")


ZeroDivisionError: division by zero

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, UnitGrid

# define the PDE
a, b = 1, 3
d0, d1 = 1, 0.1
eq = PDE(
    {
        "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
        "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
    }
)

# initialize state
grid = UnitGrid([64, 64])
u = ScalarField(grid, a, label="Field $u$")
v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")
state = FieldCollection([u, v])

# simulate the pde
sol = eq.solve(state, t_range=20, dt=1e-3)

# Plotting the final state
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the u field with an orange colormap
u_plot = ax.imshow(sol[0].data, cmap='Oranges', alpha=0.6, vmin=0, vmax=5)

# Plot the v field with a blue colormap
v_plot = ax.imshow(sol[1].data, cmap='Blues', alpha=0.6, vmin=0, vmax=5)

# Adding colorbars
cbar_u = plt.colorbar(u_plot, ax=ax, fraction=0.046, pad=0.04)
cbar_u.set_label('Compound X')

cbar_v = plt.colorbar(v_plot, ax=ax, fraction=0.046, pad=0.04)
cbar_v.set_label('Compound Y')

# Display the plot
plt.title("Combined Plot of Fields $u$ and $v$")
plt.show()

  0%|          | 0/20.0 [00:00<?, ?it/s]


KeyboardInterrupt



# Without reflective boundary conditions

In [26]:
import cv2
import os
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, UnitGrid, MemoryStorage

# Define the PDE
a, b = 1, 2
d0, d1 = 1, 0.1
eq = PDE(
    {
        "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
        "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
    }
)

# Initialize state
grid = UnitGrid([256, 256])
u = ScalarField(grid, a, label="Field $u$")
v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")
state = FieldCollection([u, v])

# Directory to save frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Create a MemoryStorage object to store the simulation state
storage = MemoryStorage()

# Simulate the PDE with storage tracker
sol = eq.solve(state, t_range=100, dt=1e-4, tracker=storage.tracker(interval=1))

# Save each state as an image
for frame_idx, (time, state) in enumerate(storage.items()):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Plot the u field with an orange colormap
    u_data = state[0].data
    u_plot = ax.imshow(u_data, cmap='Oranges', alpha=0.6, vmin=0, vmax=5)
    
    # Plot the v field with a blue colormap
    v_data = state[1].data
    v_plot = ax.imshow(v_data, cmap='Blues', alpha=0.6, vmin=0, vmax=5)
    
    # Add colorbars
    cbar_u = plt.colorbar(u_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_u.set_label('Compound X')
    cbar_v = plt.colorbar(v_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_v.set_label('Compound Y')
    
    # Set title
    plt.title(f"Compounds X and Y at t={time:.2f}")
    
    # Save the frame
    frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
    plt.savefig(frame_path)
    plt.close(fig)
    print(f"Saved frame {frame_idx} at time {time}")

# Custom directory to save the video file
video_directory = os.getcwd()
video_path = os.path.join(video_directory, 'brusselator.avi')

# Compile frames into a video
frame_rate = 24  # frames per second

# Get frame size from the first frame
first_frame = cv2.imread(os.path.join(frames_dir, 'frame_0000.png'))
if first_frame is None:
    raise ValueError("First frame not found. Check if the frames are being saved correctly.")
height, width, layers = first_frame.shape
frame_size = (width, height)

# Initialize video writer
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, frame_size)

# Write frames to video
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    if frame is None:
        print(f"Error reading frame {frame_file}. Skipping.")
        continue
    out.write(frame)

out.release()

print(f"Video saved to {video_path}")


Saved frame 0 at time 0
Saved frame 1 at time 1.0
Saved frame 2 at time 2.0
Saved frame 3 at time 3.0000000000000004
Saved frame 4 at time 4.0
Saved frame 5 at time 5.0
Saved frame 6 at time 6.0
Saved frame 7 at time 7.0
Saved frame 8 at time 8.0
Saved frame 9 at time 9.0
Saved frame 10 at time 10.0
Saved frame 11 at time 11.0
Saved frame 12 at time 12.0
Saved frame 13 at time 13.0
Saved frame 14 at time 14.0
Saved frame 15 at time 15.0
Saved frame 16 at time 16.0
Saved frame 17 at time 17.0
Saved frame 18 at time 18.0
Saved frame 19 at time 19.0
Saved frame 20 at time 20.0
Saved frame 21 at time 21.0
Saved frame 22 at time 22.0
Saved frame 23 at time 23.0
Saved frame 24 at time 24.0
Saved frame 25 at time 25.0
Saved frame 26 at time 26.0
Saved frame 27 at time 27.0
Saved frame 28 at time 28.0
Saved frame 29 at time 29.0
Saved frame 30 at time 30.0
Saved frame 31 at time 31.0
Saved frame 32 at time 32.0
Saved frame 33 at time 33.0
Saved frame 34 at time 34.0
Saved frame 35 at time 35.0

# Showing the different phases 
## Critical Point 
 a = 1, b = a^2 + 1 = 2
## Hopf Bifurcation / Oscillaratory behaviour
b > 1 + a^2
a, b = 1, 3
d0, d1 = 1, 0.

## Turing instability 
a, b = 1, 2
d0, d1 = 1, 0.0

## Steady State
b < a^2 + 1, a = 2, b = 11v1

# With reflective borders / boundary conditions

In [31]:
import cv2
import os
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, UnitGrid, MemoryStorage, CartesianGrid

# Define the PDE
a, b = 1, 2
d0, d1 = 1, 0.01
eq = PDE(
    {
        "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
        "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
    }
)

# Initialize state with reflective boundary conditions
grid = UnitGrid([64, 64], periodic=[False, False])  # Disable periodic boundaries
u = ScalarField(grid, a, label="Field $u$")
v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")
state = FieldCollection([u, v])

# Directory to save frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Create a MemoryStorage object to store the simulation state
storage = MemoryStorage()

# Simulate the PDE with storage tracker
sol = eq.solve(state, t_range=100, dt=1e-3, tracker=storage.tracker(interval=1))

# Save each state as an image
for frame_idx, (time, state) in enumerate(storage.items()):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Plot the u field with an orange colormap
    u_data = state[0].data
    u_plot = ax.imshow(u_data, cmap='Oranges', alpha=0.6, vmin=0, vmax=5)
    
    # Plot the v field with a blue colormap
    v_data = state[1].data
    v_plot = ax.imshow(v_data, cmap='Blues', alpha=0.6, vmin=0, vmax=5)
    
    # Add colorbars
    cbar_u = plt.colorbar(u_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_u.set_label('Field $u$')
    cbar_v = plt.colorbar(v_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_v.set_label('Field $v$')
    
    # Set title
    plt.title(f"Combined Plot of Fields $u$ and $v$ at t={time:.2f}")
    
    # Save the frame
    frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
    plt.savefig(frame_path)
    plt.close(fig)
    print(f"Saved frame {frame_idx} at time {time}")

# Custom directory to save the video file
video_directory = os.getcwd()
video_path = os.path.join(video_directory, 'brusselator.avi')

# Compile frames into a video
frame_rate = 24  # frames per second

# Get frame size from the first frame
first_frame = cv2.imread(os.path.join(frames_dir, 'frame_0000.png'))
if first_frame is None:
    raise ValueError("First frame not found. Check if the frames are being saved correctly.")
height, width, layers = first_frame.shape
frame_size = (width, height)

# Initialize video writer
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, frame_size)

# Write frames to video
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    if frame is None:
        print(f"Error reading frame {frame_file}. Skipping.")
        continue
    out.write(frame)

out.release()

print(f"Video saved to {video_path}")


Saved frame 0 at time 0
Saved frame 1 at time 1.0
Saved frame 2 at time 2.0
Saved frame 3 at time 3.0
Saved frame 4 at time 4.0
Saved frame 5 at time 5.0
Saved frame 6 at time 6.0
Saved frame 7 at time 7.0
Saved frame 8 at time 8.0
Saved frame 9 at time 9.0
Saved frame 10 at time 10.0
Saved frame 11 at time 11.0
Saved frame 12 at time 12.0
Saved frame 13 at time 13.0
Saved frame 14 at time 14.0
Saved frame 15 at time 15.0
Saved frame 16 at time 16.0
Saved frame 17 at time 17.0
Saved frame 18 at time 18.0
Saved frame 19 at time 19.0
Saved frame 20 at time 20.0
Saved frame 21 at time 21.0
Saved frame 22 at time 22.0
Saved frame 23 at time 23.0
Saved frame 24 at time 24.0
Saved frame 25 at time 25.0
Saved frame 26 at time 26.0
Saved frame 27 at time 27.0
Saved frame 28 at time 28.0
Saved frame 29 at time 29.0
Saved frame 30 at time 30.0
Saved frame 31 at time 31.0
Saved frame 32 at time 32.0
Saved frame 33 at time 33.0
Saved frame 34 at time 34.0
Saved frame 35 at time 35.0
Saved frame 36

# Petrischale

In [34]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, UnitGrid, MemoryStorage

# Constants
RESOLUTION = 256  # Resolution of the grid
RADIUS = RESOLUTION // 2  # Radius of the circular region (half of the resolution)
FRAME_RATE = 24  # Frames per second for the video
T_MAX = 50  # Maximum simulation time
DT = 1e-3  # Time step

# Define the PDE
a, b = 1, 3
d0, d1 = 1, 0.1
eq = PDE(
    {
        "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
        "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
    }
)

# Initialize state with reflective boundary conditions
grid = UnitGrid([RESOLUTION, RESOLUTION], periodic=[False, False])

# Create initial fields
u = ScalarField(grid, a, label="Field $u$")
v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")

# Create a circular mask for the Dirichlet boundary condition
center = (grid.shape[0] // 2, grid.shape[1] // 2)
Y, X = np.ogrid[:grid.shape[0], :grid.shape[1]]
dist_from_center = np.sqrt((X - center[1])**2 + (Y - center[0])**2)
circular_mask = dist_from_center <= RADIUS

# Apply the mask to enforce the Dirichlet boundary conditions
u.data[~circular_mask] = 0
v.data[~circular_mask] = 0

# Create a state collection
state = FieldCollection([u, v])

# Directory to save frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Create a MemoryStorage object to store the simulation state
storage = MemoryStorage()

# Simulate the PDE with storage tracker
sol = eq.solve(state, t_range=T_MAX, dt=DT, tracker=storage.tracker(interval=1))

# Save each state as an image
for frame_idx, (time, state) in enumerate(storage.items()):
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Apply the circular mask to the data
    u_data = np.ma.masked_where(~circular_mask, state[0].data)
    v_data = np.ma.masked_where(~circular_mask, state[1].data)
    
    # Plot the u field with an orange colormap
    u_plot = ax.imshow(u_data, cmap='Oranges', alpha=0.6, vmin=0, vmax=5)
    
    # Plot the v field with a blue colormap
    v_plot = ax.imshow(v_data, cmap='Blues', alpha=0.6, vmin=0, vmax=5)
    
    # Add colorbars
    cbar_u = plt.colorbar(u_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_u.set_label('Field $u$')
    cbar_v = plt.colorbar(v_plot, ax=ax, fraction=0.046, pad=0.04)
    cbar_v.set_label('Field $v$')
    
    # Set title
    plt.title(f"Combined Plot of Fields $u$ and $v$ at t={time:.2f}")
    
    # Save the frame
    frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
    plt.savefig(frame_path)
    plt.close(fig)
    print(f"Saved frame {frame_idx} at time {time}")

# Custom directory to save the video file
video_directory = os.getcwd()
video_path = os.path.join(video_directory, 'brusselator.avi')

# Compile frames into a video
first_frame = cv2.imread(os.path.join(frames_dir, 'frame_0000.png'))
if first_frame is None:
    raise ValueError("First frame not found. Check if the frames are being saved correctly.")
height, width, layers = first_frame.shape
frame_size = (width, height)

# Initialize video writer
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), FRAME_RATE, frame_size)

# Write frames to video
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
for frame_file in frame_files:
    frame = cv2.imread(os.path.join(frames_dir, frame_file))
    if frame is None:
        print(f"Error reading frame {frame_file}. Skipping.")
        continue
    out.write(frame)

out.release()

print(f"Video saved to {video_path}")


Saved frame 0 at time 0
Saved frame 1 at time 1.0
Saved frame 2 at time 2.0
Saved frame 3 at time 3.0
Saved frame 4 at time 4.0
Saved frame 5 at time 5.0
Saved frame 6 at time 6.0
Saved frame 7 at time 7.0
Saved frame 8 at time 8.0
Saved frame 9 at time 9.0
Saved frame 10 at time 10.0
Saved frame 11 at time 11.0
Saved frame 12 at time 12.0
Saved frame 13 at time 13.0
Saved frame 14 at time 14.0
Saved frame 15 at time 15.0
Saved frame 16 at time 16.0
Saved frame 17 at time 17.0
Saved frame 18 at time 18.0
Saved frame 19 at time 19.0
Saved frame 20 at time 20.0
Saved frame 21 at time 21.0
Saved frame 22 at time 22.0
Saved frame 23 at time 23.0
Saved frame 24 at time 24.0
Saved frame 25 at time 25.0
Saved frame 26 at time 26.0
Saved frame 27 at time 27.0
Saved frame 28 at time 28.0
Saved frame 29 at time 29.0
Saved frame 30 at time 30.0
Saved frame 31 at time 31.0
Saved frame 32 at time 32.0
Saved frame 33 at time 33.0
Saved frame 34 at time 34.0
Saved frame 35 at time 35.0
Saved frame 36

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from pde import PDE, FieldCollection, ScalarField, CartesianGrid, MemoryStorage

# Constants
RESOLUTION = 128  # Resolution of the grid
FRAME_RATE = 24  # Frames per second for the video
OSCILLATION_PERIOD = 2  # Period of oscillation in seconds
NUM_OSCILLATIONS = 10  # Number of oscillations to simulate
COLOR_VMIN = 0  # Minimum value for color scaling
COLOR_VMAX = 5  # Maximum value for color scaling

# Calculate total simulation time and time step
T_MAX = NUM_OSCILLATIONS * OSCILLATION_PERIOD
DT = OSCILLATION_PERIOD / (FRAME_RATE * 2)  # Time step to get smooth frames for video

# Modes configuration
modes = [
    {
        "title": "Stable Steady State",
        "a": 1,
        "b": 1,
        "d0": 1,
        "d1": 0.1,
        "u_color": "Greens",
        "v_color": "Purples",
        "filename": "stable_steady_state.avi",
        "description": "b < a^2 + 1\nBehavior: The system reaches a stable steady state without oscillations."
    },
    {
        "title": "Oscillatory Behavior (Hopf Bifurcation)",
        "a": 1,
        "b": 3,
        "d0": 1,
        "d1": 0.1,
        "u_color": "Oranges",
        "v_color": "Blues",
        "filename": "oscillatory_behavior.avi",
        "description": "b > a^2 + 1\nBehavior: The system exhibits sustained oscillations."
    },
    {
        "title": "Pattern Formation (Turing Instability)",
        "a": 1,
        "b": 2,
        "d0": 1,
        "d1": 0.01,
        "u_color": "Reds",
        "v_color": "Greens",
        "filename": "pattern_formation.avi",
        "description": "b > a^2 + 1 and η = d1/d0 > 1 + a^2 - b/a^2\nBehavior: The system forms spatial patterns due to diffusion-driven instability."
    },
    {
        "title": "Critical Point (Threshold of Oscillations)",
        "a": 1,
        "b": 2,
        "d0": 1,
        "d1": 0.1,
        "u_color": "Purples",
        "v_color": "Oranges",
        "filename": "critical_point.avi",
        "description": "b = a^2 + 1\nBehavior: The system is at the threshold of oscillations."
    }
]

# Create results directory
results_dir = 'results'
os.makedirs(results_dir, exist_ok=True)

# Determine the next render number
render_numbers = [int(name) for name in os.listdir(results_dir) if name.isdigit()]
render_number = max(render_numbers, default=0) + 1
render_dir = os.path.join(results_dir, str(render_number))
os.makedirs(render_dir, exist_ok=True)

storage_dict = {}  # Store the MemoryStorage objects for each mode

for mode in modes:
    # Extract parameters
    title = mode["title"]
    a = mode["a"]
    b = mode["b"]
    d0 = mode["d0"]
    d1 = mode["d1"]
    u_color = mode["u_color"]
    v_color = mode["v_color"]
    filename = mode["filename"]
    description = mode["description"]

    # Define the PDE
    eq = PDE(
        {
            "u": f"{d0} * laplace(u) + {a} - ({b} + 1) * u + u**2 * v",
            "v": f"{d1} * laplace(v) + {b} * u - u**2 * v",
        }
    )

    # Initialize state with reflective boundary conditions
    RADIUS = RESOLUTION // 2
    grid = CartesianGrid([[-RADIUS, RADIUS], [-RADIUS, RADIUS]], [RESOLUTION, RESOLUTION], periodic=False)

    # Create initial fields
    u = ScalarField(grid, a, label="Field $u$")
    v = b / a + 0.1 * ScalarField.random_normal(grid, label="Field $v$")

    # Create a circular mask for the Dirichlet boundary condition
    center = (grid.shape[0] // 2, grid.shape[1] // 2)
    Y, X = np.ogrid[:grid.shape[0], :grid.shape[1]]
    dist_from_center = np.sqrt((X - center[1])**2 + (Y - center[0])**2)
    circular_mask = dist_from_center <= RADIUS

    # Apply the mask to enforce the Dirichlet boundary conditions
    u.data[~circular_mask] = 0
    v.data[~circular_mask] = 0

    # Create a state collection
    state = FieldCollection([u, v])

    # Directory to save frames
    frames_dir = os.path.join(render_dir, f'frames_{title.replace(" ", "_").lower()}')
    os.makedirs(frames_dir, exist_ok=True)

    # Create a MemoryStorage object to store the simulation state
    storage = MemoryStorage()

    # Simulate the PDE with storage tracker
    sol = eq.solve(state, t_range=T_MAX, dt=DT, tracker=storage.tracker(interval=1))
    storage_dict[title] = list(storage.items())  # Store the items in a list

    # Save each state as an image
    for frame_idx, (time, state) in enumerate(storage_dict[title]):
        fig, ax = plt.subplots(figsize=(8, 8))
        
        # Apply the circular mask to the data
        u_data = np.ma.masked_where(~circular_mask, state[0].data)
        v_data = np.ma.masked_where(~circular_mask, state[1].data)
        
        # Plot the u field with the specified colormap
        u_plot = ax.imshow(u_data, cmap=u_color, alpha=0.6, vmin=COLOR_VMIN, vmax=COLOR_VMAX, extent=[-RADIUS, RADIUS, -RADIUS, RADIUS])
        
        # Plot the v field with the specified colormap
        v_plot = ax.imshow(v_data, cmap=v_color, alpha=0.6, vmin=COLOR_VMIN, vmax=COLOR_VMAX, extent=[-RADIUS, RADIUS, -RADIUS, RADIUS])
        
        # Add colorbars with labels below
        cbar_u = plt.colorbar(u_plot, ax=ax, fraction=0.046, pad=0.08)
        cbar_u.ax.set_ylabel('Compound X', labelpad=10)
        
        cbar_v = plt.colorbar(v_plot, ax=ax, fraction=0.046, pad=0.14)
        cbar_v.ax.set_ylabel('Compound Y', labelpad=10)
        
        # Set title
        plt.title(title)
        
        # Set axis labels
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        
        # Add parameters text in a box at the bottom left within the plot
        params_text = f'a = {a}\nb = {b}\nd0 = {d0}\nd1 = {d1}'
        ax.text(-RADIUS, -RADIUS, params_text, ha='left', va='bottom', 
                bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))
        
        # Save the frame
        frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
        plt.savefig(frame_path)
        plt.close(fig)
        print(f"Saved frame {frame_idx} at time {time}")

    # Custom directory to save the video file
    video_path = os.path.join(render_dir, filename)

    # Compile frames into a video
    first_frame = cv2.imread(os.path.join(frames_dir, 'frame_0000.png'))
    if first_frame is None:
        raise ValueError("First frame not found. Check if the frames are being saved correctly.")
    height, width, layers = first_frame.shape
    frame_size = (width, height)

    # Initialize video writer
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), FRAME_RATE, frame_size)

    # Write frames to video
    frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(frames_dir, frame_file))
        if frame is None:
            print(f"Error reading frame {frame_file}. Skipping.")
            continue
        out.write(frame)

    out.release()

    print(f"Video saved to {video_path}")

# Create a video with a 2x2 grid of the four modes
grid_video_path = os.path.join(render_dir, 'overview_phases.avi')
out_grid = cv2.VideoWriter(grid_video_path, cv2.VideoWriter_fourcc(*'XVID'), FRAME_RATE, (2 * width, 2 * height))

num_frames = len(storage_dict[modes[0]["title"]])

for frame_idx in range(num_frames):
    combined_frame = np.zeros((2 * height, 2 * width, 3), dtype=np.uint8)
    
    for i, mode in enumerate(modes):
        frames_dir = os.path.join(render_dir, f'frames_{mode["title"].replace(" ", "_").lower()}')
        frame_path = os.path.join(frames_dir, f'frame_{frame_idx:04d}.png')
        frame = cv2.imread(frame_path)
        
        if frame is None:
            print(f"Error reading frame {frame_path}. Skipping.")
            continue
        
        if i == 0:
            combined_frame[0:height, 0:width] = frame
        elif i == 1:
            combined_frame[0:height, width:2 * width] = frame
        elif i == 2:
            combined_frame[height:2 * height, 0:width] = frame
        elif i == 3:
            combined_frame[height:2 * height, width:2 * width] = frame

    out_grid.write(combined_frame)

out_grid.release()

print(f"Overview video saved to {grid_video_path}")


ImportError: cannot import name 'get_cmap' from 'matplotlib.cm' (c:\Users\Ott\Brusselator\.venv\Lib\site-packages\matplotlib\cm.py)